# Spatial Operations
stough 202-

To this point we have generally thought of images as collections of **independent** pixels, organized in a grid for sure but not really thought of as an ordered collection. We have heavily considered the histograms of images for use in contrast enhancement and compression, where histogramming completely destroys the spatial relationships that these pixels have with one another. In considering entropy in fact, we modeled our image as a sequence of **independent and identically distributed (iid)** symbols sampled from the histogram.

We did hint at the importance of **spatial coherence**, which is exactly the way in which images are not iid sequences. We saw that **predictive coding**, using neighbors of a pixel to predict that pixel and then encoding only the error of prediction, leads to much more highly compressible signals than the images themselves. Were pixels iid, then such an encoding would be of no benefit. If our power transform is an example of a single-pixel operation (or point transform), then predictive coding is an example of a **spatial or neighborhood operation**.

**Spatial operations** are operations that we apply to the pixels of an image that take into account the neighborhood of a pixel. Here we're going to look at some spatial operations using [`scipy.ndimage.correlate`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.correlate.html). 

## Imports
May be a warning about ufunc size.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For spatial filtering/operations
from scipy.ndimage import (correlate,
                           convolve)

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

## Review Predictive Coding
Instead of directly using `np.diff`, we can use correlate 

In [ ]:
I = plt.imread('../dip_pics/canyon.jpg')
vis_hists(I)
print(arr_info(I))

In [ ]:
J = I[...,0]
vis_hists(J)

In [ ]:
h = np.array([-1, 1], ndmin=2).astype('int16')

In [ ]:
h

In [ ]:
arr_info(h)

In [ ]:
Jf = correlate(J.astype('int16'), h, mode='constant', cval=0)
arr_info(Jf)

In [ ]:
vis_hists(Jf)

## Unsharp Masking
Using a blur to actually sharpen the edges in an image.

In [ ]:
h = np.ones((5,5))/25.0
h

In [ ]:
Jf = correlate(J.astype('float'), h, mode='constant', cval=0)

In [ ]:
vis_pair(J, Jf, cmap='gray', second_title="Blurred")

In [ ]:
arr_info(Jf)

In [ ]:
arr_info(J)

In [ ]:
D = J - Jf
vis_hists(D)

In [ ]:
vis_pair(J, D, cmap='gray')

In [ ]:
Jsharp = J + 2*D 
# vis_pair(J, Jsharp, cmap='gray') # results in less contrast due to out-of-range pixels messing up display.
vis_pair(J, np.clip(Jsharp,0,255), cmap='gray', second_title="Sharpened")

In [ ]:
arr_info(Jsharp)

In [ ]:
plt.figure()
plt.plot(J[193, 250:270])
plt.plot(np.clip(Jsharp,0,255)[193, 250:270])

## Gradient maps
An alternative filter design can give us locally computed vertical and horizontal *edginess* if you will. These first order derivative images in $x,y$ can allow us to compute the gradient map of an image. Basically the edginess of every pixel.

In [ ]:
I = plt.imread('../dip_pics/canyon.jpg')
vis_hists(I)
print(arr_info(I))

In [ ]:
h = np.ones((5,5))/25.0
h

In [ ]:
h = np.array([[1, 0, -1],[2, 0, -2],[1, 0, -1]])

In [ ]:
h

In [ ]:
plt.figure()
plt.imshow(h, cmap='gray')

In [ ]:
I = I/I.max()

In [ ]:
vis_hists(I)

In [ ]:
Gx = correlate(I[...,0], h, mode='nearest')

In [ ]:
vis_pair(J, Gx, cmap='gray', second_title="GX")

In [ ]:
Gy = correlate(I[...,0], h.transpose(), mode='nearest')

In [ ]:
vis_pair(Gx, Gy, cmap='gray', first_title='Gx', second_title='Gy')

In [ ]:
G = np.sqrt(Gx**2 + Gy**2)
# G = Gx**2 + Gy**2

In [ ]:
arr_info(G)

In [ ]:
vis_pair(J, G, cmap='gray', second_title='Grad Mag')

In [ ]:
h = np.array([[-1, -1, -1],[-1, 8, -1],[-1, -1, -1]])

In [ ]:
h

In [ ]:
J = correlate(I[...,0], h, mode='nearest')

In [ ]:
vis_pair(I, np.abs(J), cmap='gray')